In [1]:
from alpaca_trade_api import REST, Stream
import csv
import pandas as pd
from my_secrets import APCA_API_KEY_ID, APCA_API_SECRET_KEY

In [2]:
# Function to calculate the moving averages
def calculate_moving_averages(prices, short_window=5, long_window=20):
    short_ma = sum(prices[-short_window:]) / short_window
    long_ma = sum(prices[-long_window:]) / long_window
    return short_ma, long_ma

In [3]:
# Function to generate a signal based on moving average crossover
def generate_signal(prices, short_window=5, long_window=20):
    if len(prices) < long_window:
        return None  # Not enough data to generate a signal

    short_ma, long_ma = calculate_moving_averages(prices, short_window, long_window)

    if short_ma > long_ma:
        return 'BUY'
    elif short_ma < long_ma:
        return 'SELL'
    else:
        return None

# Main loop to simulate live data and execute trades based on signals

In [4]:
def main():
    api = REST(APCA_API_KEY_ID, APCA_API_SECRET_KEY)

    # Define trading parameters
    symbol = 'BTCUSD'
    short_window = 5
    long_window = 20
    initial_cash = 1000

    # Get historical data for initialization
    historical_data = api.get_barset(symbol, '1Min', limit=long_window+1).df[symbol]

    # Initialize prices list with historical data
    prices = list(historical_data['close'])

    # Initialize the portfolio with cash and no positions
    cash = initial_cash
    position = 0

    for BTC in Stream(APCA_API_KEY_ID, APCA_API_SECRET_KEY, raw_data=False).subscribe_crypto_bars():
        print(BTC)

        # Update prices list with the latest data
        prices.append(BTC.close)

        # Generate a signal based on moving average crossover
        signal = generate_signal(prices, short_window, long_window)

        if signal == 'BUY' and cash > 0:
            # Place a buy order
            quantity = cash / BTC.close  # Allocate all available cash to buy BTC
            api.submit_order(
                symbol=symbol,
                qty=quantity,
                side='buy',
                type='market',
                time_in_force='gtc'
            )
            cash = 0
            position += quantity

        elif signal == 'SELL' and position > 0:
            # Place a sell order
            api.submit_order(
                symbol=symbol,
                qty=position,
                side='sell',
                type='market',
                time_in_force='gtc'
            )
            cash += position * BTC.close
            position = 0

        # Remove the oldest price to keep the list length consistent with the window sizes
        prices.pop(0)


In [5]:
if __name__ == "__main__":
    main()

AttributeError: 'REST' object has no attribute 'get_barset'